# Actividad Integradora

Alejandra Cabrera Ruiz

A01704463

**Puntos a considerar**

- La semilla para generación de números aleatorios será 67890.
- El almacén es 20x20 celdas.
- Al inicio de la simulación, tu solución deberá colocar 200 cajas repartidas en grupos de 1 a 3 cajas en posiciones aleatorias.
- Todos los robots empiezan en posiciones aleatorias vacías. Y, sólo puede haber un robot por celda.
- La simulación termina cuando todas las cajas se encuentra apiladas en pilas de exactamente 5 cajas.

**¿Que debes entregar?**

Un cuaderno de Jupyter Notebook conteniendo un reporte de la actividad. El cuaderno deberá contener:
- Código fuente documentado.
- Descripción detallada de la estrategia y los mecanismos utilizados en tu solución.
- Una visualización que permita ver los diferentes pasos de la simulación.
- El número de pasos necesarios para terminar la simulación.
- ¿Existe una forma de reducir el número de pasos utilizados? Si es así, ¿cuál es la estrategia que se tendría en implementar?


In [8]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import pandas as pd
import random
from random import shuffle

import time 
import datetime

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [3]:
class RobotAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.has_box = False
    
    def move(self):
        possible_moves = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = self.random.choice(possible_moves)
        self.model.grid.move_agent(self, new_position)
    
    def pickup_box(self):
        x, y = self.pos
        contents = self.model.grid.get_cell_list_contents([(x, y)])

        for content in contents:
            if content == "box" and not self.has_box:
                self.has_box = True
                return True
        return False
    
    def drop_box(self):
        x, y = self.pos
        contents = self.model.grid.get_cell_list_contents([(x, y)])

        for content in contents:
            if content == "box" and self.has_box:
                self.has_box = False
                return True
        return False

In [11]:
class BoxAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

In [12]:
class BoxStackingModel(Model):
    def __init__(self, width, height, num_boxes, num_robots, seed):
        random.seed(seed)
        self.num_robots = num_robots
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        for i in range(num_boxes):
            x, y = self.random.randrange(width), self.random.randrange(height)
            num_boxes_in_stack = self.random.randint(1, 4)
            for j in range(num_boxes_in_stack):
                box = BoxAgent(f"box_{i}_{j}", self)
                self.grid.place_agent(box, (x, y))

        # Create robots
        for i in range(num_robots):
            x, y = self.random_empty_position()
            robot = RobotAgent(f"robot_{i}", self)
            self.grid.place_agent(robot, (x, y))
            self.schedule.add(robot)



        self.datacollector = DataCollector(
            agent_reporters={"Has Box": "has_box"}
        )

    def random_empty_position(self):
    # Get a random empty position in the grid
        while True:
            x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
            cell_contents = self.grid.get_cell_list_contents((x, y))
            if len(cell_contents) == 0 and all(not isinstance(agent, RobotAgent) for agent in cell_contents):
                return x, y

    def stack_boxes(self):
        stack_limit = 40  # Número máximo de pilas que queremos alcanzar
        stacks = 0  # Contador de pilas creadas

        # Recorremos todas las celdas en la cuadrícula
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                contents = self.grid.get_cell_list_contents([(x, y)])
                robots = [agent for agent in contents if isinstance(agent, RobotAgent)]
                
                # Si hay al menos un robot en la celda
                if robots:
                    robot = robots[0]  # Suponiendo que solo hay un robot por celda
                    if len(contents) >= 5 and not robot.has_box:
                        # Obtener las cajas disponibles para que el robot las recoja
                        available_boxes = [agent for agent in contents if agent == "box" and not robot.has_box]

                        # Elegir aleatoriamente si cargar o apilar las cajas
                        if available_boxes:
                            shuffle(available_boxes)
                            box_to_pick = available_boxes[0]

                            # El robot carga la caja
                            robot.pickup_box()

                            # Verificar si se ha creado una pila
                            contents_after_pickup = self.grid.get_cell_list_contents([(x, y)])
                            if len(contents_after_pickup) >= 5 and contents_after_pickup.count("box") == 5:
                                stacks += 1

                                # Detener la simulación si hemos alcanzado el límite de pilas
                                if stacks >= stack_limit:
                                    return

                            # El robot coloca la caja en otra posición aleatoria
                            new_x, new_y = self.random_empty_position()
                            robot.put_down_box()
                            box_to_pick.move_to((new_x, new_y))

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        self.stack_boxes()

In [9]:
# Función para la visualización del modelo
def visualize_model(model):
    fig, ax = plt.subplots(figsize=(8, 8))

    # Función de actualización para la animación
    def update(frame):
        ax.clear()
        ax.set_title(f"Step: {frame}")
        ax.set_xlim(0, model.grid.width)
        ax.set_ylim(0, model.grid.height)

        # Dibujar cajas y robots
        for agent in model.schedule.agents:
            if isinstance(agent, RobotAgent) and agent.has_box:
                ax.plot(agent.pos[0] + 0.5, agent.pos[1] + 0.5, 'ro', markersize=10)
            elif isinstance(agent, BoxAgent):
                ax.plot(agent.pos[0] + 0.5, agent.pos[1] + 0.5, 'bo', markersize=8)

    ani = animation.FuncAnimation(fig, update, frames=None, repeat=False, blit=False)
    return ani


In [ ]:
WIDTH = 20
HEIGHT = 20
NUM_BOXES = 200
NUM_ROBOTS = 5
SEED = 67890

steps = 0

model = BoxStackingModel(WIDTH, HEIGHT, NUM_BOXES, NUM_ROBOTS, SEED)
while any(agent.has_box for agent in model.schedule.agents):
    model.step()
    steps += 1

print(f"Steps: {steps}")

In [14]:
animation = visualize_model(model)
animation

C:\Users\aleja\AppData\Local\Temp\ipykernel_35256\913597647.py:19: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update, frames=None, repeat=False, blit=False)


¿Existe una forma de reducir el número de pasos utilizados? Si es así, ¿cuál es la estrategia que se tendría en implementar?

Sí, probablemente si los robots se pudieran comunicar entre ellos sería muchísimo más rápido, añadir un floor y un floor conocido como en el reto. Del mismo modo aleatoriamente marcaría 40 cajas como target sin decirle al robot donde marcarlas. Las primeras 40 cajas que encuentre las marca como target y las demás las apila en esas (tal vez implementó esto). Igual podríamos dividir el espacio y que cada robot solo se moviera en un 1/5 del espacio lo que optimizaría el tiempo que les toma apilar nada más su espacio asignado.